In [4]:
#导入必须的包
from langchain.document_loaders import UnstructuredExcelLoader,Docx2txtLoader,PyPDFLoader,TextLoader
from langchain.text_splitter import  CharacterTextSplitter
from langchain.vectorstores import  Chroma
#导入聊天所需的模块
from langchain_openai import ChatOpenAI
# from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_community.embeddings import DashScopeEmbeddings
#向量化
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
from typing import List
import numpy as np

class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        # 编码文本列表，返回嵌套列表形式的向量
        return self.model.encode(texts).tolist()

    def embed_query(self, text: str) -> List[float]:
        # 编码单个文本，返回列表形式的向量
        return self.model.encode(text).tolist()

tongyiKey = "sk-6ac9a597e95c46bca363c63eb34ac770"
dsKey = "sk-a51b195199f844cc9392fbbf105239cc"
tongyiUrl = "https://dashscope.aliyuncs.com/compatible-mode/v1"
dsUrl = "https://api.deepseek.com"
#定义chatdoc
class ChatDoc():
    def __init__(self):
        self.doc = None
        self.splitText = [] #分割后的文本
        # self.template = [
        #     ("system","你是一个处理文档的秘书,你会根据下面提供的上下文内容来继续回答问题.\n 上下文内容\n {context} \n"),
        #     ("human","你好！"),
        #     ("ai","你好"),
        #     ("human","{question}"),
        # ]
        self.template = [
            ("system",
             "你是一名历史教学助手，擅长根据教材内容设计教学题目。请严格遵循以下要求：\n"
             "1. 基于提供的教学材料生成题目，包含选择题/简答题/材料分析题\n"
             "2. 标注题目难度（基础/进阶/拓展）\n"
             "3. 确保史实准确，重要事件需标注时间\n"
             "4. 答案需包含解析和对应教材页码\n"
             "当前教学材料：\n{context}\n"),

            ("human", "请为《古代丝绸之路》这一课设计3道选择题和1道材料分析题"),

            ("ai",
             "【选择题】\n"
             "1.（基础）张骞首次出使西域的主要目的是？\n"
             "A. 开辟贸易路线  B. 联合大月氏  C. 传播佛教  D. 寻找汗血马\n"
             "答案：B（教材P45）\n\n"
             "【材料分析】\n"
             "阅读敦煌壁画记载的商队文书...\n"
             "问题：分析丝绸之路上的文化交流特点\n"
             "评分要点：物质交流与精神传播并重（教材P48）"),

            ("human", "{question}"),
        ]
        self.prompt = ChatPromptTemplate.from_messages(self.template)

        # # 初始化阿里云文本向量模型
        # self.embeddings = DashScopeEmbeddings(
        #     model="text-embedding-v3",
        #     dashscope_api_key="sk-6ac9a597e95c46bca363c63eb34ac770",
        # )
        #第二个向量化方式
        self.embeddings = SentenceTransformerEmbeddings("paraphrase-multilingual-MiniLM-L12-v2")
    def getFile(self):
        doc = self.doc
        loaders = {
            "txt":TextLoader,
            "docx":Docx2txtLoader,
            "pdf":PyPDFLoader,
            "xlsx":UnstructuredExcelLoader,
        }
        file_extension = doc.split(".")[-1]
        loader_class = loaders.get(file_extension)
        if loader_class:
            try:
                if file_extension == "txt":
                    loader = TextLoader(doc, encoding="utf-8")
                else:
                    loader = loader_class(doc)
                # loader = TextLoader(doc, encoding="utf-8")
                # loader = loader_class(doc)
                text = loader.load()
                return text
            except Exception as e:
                print(f"Error loading {file_extension} files:{e}")
        else:
             print(f"Unsupported file extension: {file_extension}")
             return  None

    #处理文档的函数
    def splitSentences(self):
        full_text = self.getFile() #获取文档内容
        if full_text != None:
            #对文档进行分割
            text_split = CharacterTextSplitter(
                chunk_size=150,
                chunk_overlap=20,
            )
            texts = text_split.split_documents(full_text)
            self.splitText = texts
            # 向量存储
            self.db = self.embeddingAndVectorDB(texts)

    #向量化与向量存储
    def embeddingAndVectorDB(self,texts):
        # db = Chroma.from_documents(texts, self.embeddings, collection_name="example_collection", persist_directory="chroma_db_history")
        db = Chroma.from_documents(
            documents=texts,
            embedding=self.embeddings,  # 传入 Embeddings 对象
            collection_name="example_collection",
            persist_directory="chroma_db_history_1"
        )
        # 持久化数据库
        db.persist()
        return db

    #提问并找到相关的文本块
    def askAndFindFiles(self,question):
        return self.db.similarity_search(query=question, k=3)  # 返回最相关的3个文档

    #用自然语言和文档聊天
    def chatWithDoc(self,question):
        _content = ""
        context = self.askAndFindFiles(question)
        # print("检索的内容是" + str(context))
        for i in context:
            _content += i.page_content

        messages = self.prompt.format_messages(context=_content,question=question)
        # print(messages)
        llm = ChatOpenAI(
            model="deepseek-chat",
            temperature=0,
            api_key=dsKey,
            base_url=dsUrl
            )
        return llm.invoke(messages)

In [6]:
chat_doc = ChatDoc()
chat_doc.doc = "docs/【人教版】高中必修 中外历史纲要(下)历史电子课本.pdf"
chat_doc.splitSentences()

In [10]:
mycontent = chat_doc.chatWithDoc("请为资本主义世界殖民体系设计2道进阶难度题目")
print(mycontent.content)

### **进阶难度题目（资本主义世界殖民体系）**  

#### **1. 选择题（进阶）**  
19世纪末20世纪初，资本主义世界殖民体系最终形成的主要标志是（  ）  
A. 工业革命在欧洲的完成  
B. 世界领土被列强瓜分完毕  
C. 国际联盟的成立  
D. 亚非拉民族独立运动兴起  

**答案：B**  
**解析：** 19世纪末20世纪初，随着主要资本主义国家向帝国主义阶段过渡，列强加紧殖民扩张，最终将世界瓜分完毕，标志着资本主义世界殖民体系的最终形成。（教材P76）  

---  

#### **2. 材料分析题（进阶）**  
**材料：**  
> “1880年时，我们所面对的不完全是一个单一的世界，而是一个由两部分所合成的全球体系：一部分是已开发的、具有主宰性的、富有的；另一部分是落后的、依赖的、贫穷的……”  

**问题：**  
结合所学知识，分析19世纪末资本主义世界殖民体系的特点及其影响。  

**参考答案：**  
1. **特点：**  
   - 世界被划分为少数发达资本主义国家（如英、法、德）和广大殖民地半殖民地国家（如非洲、亚洲）。  
   - 列强通过政治控制、经济剥削和军事侵略维持殖民统治。  
   - 资本主义世界市场最终形成，全球联系加强。  

2. **影响：**  
   - **对殖民国家：** 促进资本积累，推动工业化发展，但也加剧列强间的矛盾（如英德竞争）。  
   - **对殖民地：** 遭受掠夺和压迫，经济畸形发展，民族工业受压制。  
   - **对世界：** 全球联系加强，但形成不平等的国际秩序，为20世纪的冲突（如一战）埋下隐患。  

（教材P76-77）


In [9]:
mycontent = chat_doc.chatWithDoc("讲一下明治维新保留了封建残余这个观点，并为该观点寻找佐证的史料并给出具体例子，多分点举几个事例，比如征兵制度。,最后史料后面要写出处,输出格式为：观点：\n 史料佐证：分史料和例子")
print(mycontent.content)

**观点：**  
明治维新虽然推动了日本的现代化，但仍保留了大量的封建残余，这些残余体现在政治、经济、军事和社会等多个方面。  

**史料佐证：**  

1. **政治制度：华族与藩阀政治**  
   - **史料**：明治政府于1884年颁布《华族令》，将旧公卿、大名等封建贵族改列为“华族”，赋予其特权地位，并设立贵族院（《明治宪法》第34条）。  
   - **例子**：伊藤博文、山县有朋等长州、萨摩藩出身的政治家长期垄断政权，形成“藩阀政治”，阻碍了真正的民主化进程。  
   - **出处**：日本《明治宪法》（1889年），《华族令》（1884年）。  

2. **经济结构：地主制度与寄生地主**  
   - **史料**：1873年《地租改正条例》承认土地私有，但未彻底废除封建土地关系，导致大量农民沦为佃农。  
   - **例子**：许多旧武士和商人成为新兴地主，剥削佃农，形成半封建的寄生地主经济。  
   - **出处**：日本《地租改正条例》（1873年）。  

3. **军事制度：征兵制与武士特权残余**  
   - **史料**：1873年《征兵令》规定全民皆兵，但军官阶层仍多由旧武士（士族）垄断。  
   - **例子**：陆军大将山县有朋、海军大将东乡平八郎等均出身旧武士阶层，军队内部等级森严，带有封建色彩。  
   - **出处**：日本《征兵令》（1873年）。  

4. **社会文化：武士道精神与等级观念**  
   - **史料**：明治政府宣扬“忠君爱国”，将武士道精神改造为国家意识形态（如《教育敕语》1890年）。  
   - **例子**：社会仍强调“士农工商”的等级观念，平民与华族、士族之间存在明显差别。  
   - **出处**：日本《教育敕语》（1890年）。  

5. **法律体系：家族制度与家长权**  
   - **史料**：1898年《明治民法》规定“家制度”，家长（户主）对家族成员具有绝对权威。  
   - **例子**：女性在家庭中地位低下，婚姻和财产权受家长控制，延续了封建家族伦理。  
   - **出处**：日本《明治民法》（1898年）。  

**结论：**  
明治维新的改革并未彻底清除封建因素，反而在现代化过程中保留了旧势力的特权，这些残余影响了日本后来的